In [12]:
import numpy as np

In [13]:
def GetHam3(LabelNumber, g):
    # Define Pauli matrices
    sx = np.array([[0, 1], [1, 0]], dtype=complex)
    sy = np.array([[0, -1j], [1j, 0]], dtype=complex)
    sz = np.array([[1, 0], [0, -1]], dtype=complex)

    # Determine the number of qubits
    nq = int(np.floor(np.log2(LabelNumber)) + 1)

    # Initialize state at ground state |0...0>
    QS = {'psi0': np.zeros(2**nq, dtype=complex)}
    QS['psi0'][0] = 1  

    # Measurement Operators for labels
    QS['M'] = []
    EYE1 = np.eye(2**nq, dtype=complex)
    for k in range(LabelNumber):
        ek = EYE1[:, k]
        QS['M'].append(np.outer(ek, ek.conj()))

    # Control Hamiltonians
    QS['Hc'] = []
    for k in range(nq):
        EYE1 = np.eye(2**max(0, k - 1), dtype=complex)
        EYE2 = np.eye(2**max(0, nq - k), dtype=complex)
        QS['Hc'].append(np.kron(np.kron(EYE1, sx), EYE2))
        QS['Hc'].append(np.kron(np.kron(EYE1, sy), EYE2))

    # N operators
    QS['N'] = []
    for k in range(nq):
        EYE1 = np.eye(2**max(0, k - 1), dtype=complex)
        EYE2 = np.eye(2**max(0, nq - k), dtype=complex)
        QS['N'].append(np.kron(np.kron(EYE1, sz), EYE2))

    # Drift Hamiltonian
    QS['Delta'] = []
    QS['H0'] = np.zeros((2**nq, 2**nq), dtype=complex)
    for a in range(nq - 1):
        for b in range(a + 1, nq):
            if g[a, b] != 0:
                EYE1 = np.eye(2**max(0, a - 1), dtype=complex)
                EYE2 = np.eye(2**max(0, b - a - 1), dtype=complex)
                EYE3 = np.eye(2**max(0, nq - b), dtype=complex)
                temp = np.kron(EYE1, sz)
                temp = np.kron(temp, EYE2)
                temp = np.kron(temp, sz)
                temp = np.kron(temp, EYE3)
                QS['H0'] += g[a, b] * temp

    return QS

In [14]:
nq = 4
g_c = [1.5, 2, 2.7, 2.3, 3]
g = 2 * np.pi * np.diag(g_c[:nq - 1], k=1)

LabelNumber = 10  # Replace with actual LabelNumber value
QS = GetHam3(LabelNumber, g)

ValueError: operands could not be broadcast together with shapes (16,16) (32,32) (16,16) 